In [13]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import col, count, avg, stddev, lag

# Initialising Spark session and loading data
spark = SparkSession.builder.appName("Time-Based Aggregations for Fraud Detection").getOrCreate()
data_path = "../data/raw/creditcard.csv"
df = spark.read.csv(data_path, header=True, inferSchema=True)




24/11/12 19:01:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 19:01:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 19:01:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 19:01:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 19:01:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 19:01:42 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 1

+----+------------------+-------------------+------------------+-------------------+-------------------+-------------------+--------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+------+-----+------------------------+
|Time|                V1|                 V2|                V3|                 V4|                 V5|                 V6|                  V7|                 V8|                V9|                V10|               V11|               V12|                V13|                V14|                V15|                V16|                 V17|                V18|                V19|             

### 2.1 Number of Transactions for a Time Window
**Why:** High transaction counts within a short period can indicate suspicious behavior.

**How:** Using a rolling window to count transactions within defined time intervals.

#### Time window 1 hour

In [16]:
# Ordering Time by ascending to be sure
df = df.orderBy("Time")

# Defining a 1-hour rolling window based on Time in seconds
one_hour_window = Window.orderBy("Time").rangeBetween(-3600, 0)

# Calculating the count of transactions in the last 1 hour
df = df.withColumn("Transactions_Last_1_Hour", count("Amount").over(one_hour_window))

# checking the result
df.show()

24/11/12 20:22:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 20:22:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 20:22:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 20:22:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 20:22:10 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 20:22:13 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 2

+----+------------------+-------------------+------------------+-------------------+-------------------+-------------------+--------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+------+-----+------------------------+------------------------+
|Time|                V1|                 V2|                V3|                 V4|                 V5|                 V6|                  V7|                 V8|                V9|                V10|               V11|               V12|                V13|                V14|                V15|                V16|                 V17|                V18|        

#### Time window 5 mins

In [15]:
# Ordering Time by ascending to be sure
df = df.orderBy("Time")

# Defining a 5-mins rolling window based on Time in seconds
five_mins_window = Window.orderBy("Time").rangeBetween(-300, 0)

# Calculating the count of transactions in the last 5 mins
df = df.withColumn("Transactions_Last_5_Mins", count("Amount").over(five_mins_window))

### 2.2 Average Transaction Amount

**Why:** Sudden increases in average transaction amounts can signify abnormal spending patterns, which are often associated with fraud.

#### Average amount in last 1 hour

In [17]:
from pyspark.sql.functions import avg

# Rolling average amount within the last hour
df = df.withColumn("Avg_Amount_Last_1_Hour", avg("Amount").over(one_hour_window))

#### Average amount in last 5 mins

In [18]:
# Rolling average amount within the last hour
df = df.withColumn("Avg_Amount_Last_5_Mins", avg("Amount").over(five_mins_window))

### 2.3 High Value Transaction Flag

**Why:** Fraudulent transactions are often either very high or very low in value. This binary flag helps the model recognise when transaction values deviate significantly from the norm.

In [19]:
# Flagging transactions above a certain threshold (e.g., $2000)
df = df.withColumn("High_Amount", (col("Amount") > 2000).cast("integer"))

### 2.4 Rolling Standard Deviation of Transaction Amounts

**Why:** High variance in transaction amounts over a short period can indicate erratic spending, which is often associated with fraud attempts.

#### Time window 1 hour

In [20]:
from pyspark.sql.functions import stddev

# Rolling standard deviation of transaction amount in the last hour
df = df.withColumn("Stddev_Amount_Last_1_Hour", stddev("Amount").over(one_hour_window))

#### Time window 5 mins

In [21]:
# Rolling standard deviation of transaction amount in the last 5 mins
df = df.withColumn("Stddev_Amount_Last_5_Mins", stddev("Amount").over(five_mins_window))

In [22]:
# Specify the output path where you want to save the CSV
output_path = "../data/test"

# Save the DataFrame as a CSV file
df.write.csv(output_path, header=True, mode="overwrite")


24/11/12 20:46:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 20:46:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 20:46:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 20:46:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 20:46:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 20:46:54 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/11/12 2